# AI Platform with TFX

In [2]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import external_input
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx import components
import tensorflow_model_analysis as tfma

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

/opt/conda/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


In [3]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.1.0
TFX version: 0.21.4


In [4]:
tfx.__path__[0]

'/opt/conda/lib/python3.7/site-packages/tfx'

### Census Data

In [5]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the Census data example.
_binary_root = "gs://crazy-hippo-01/tfx/binary_classification/raw/"

# This is the path where your model will be pushed for serving.
_serving_model_dir = 'gs://crazy-hippo-01/tfx/binary_classification/serving_model/'

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

### Interactive Context

In [6]:
context = InteractiveContext()

## 1. ExampleGen

### Census Data

In [7]:
example_gen = CsvExampleGen(input=external_input(_binary_root))

In [8]:
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data gs://crazy-hippo-01/tfx/binary_classification/raw/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/CsvExampleGen/examples/1, id: 2)]
        ))

### Visualizations

In [9]:
%%skip_for_export

artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/CsvExampleGen/examples/1
This cell will be skipped during export to pipeline.


In [10]:
%%skip_for_export

train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  print(example)

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 39
      }
    }
  }
  feature {
    key: "capital_gain"
    value {
      int64_list {
        value: 2174
      }
    }
  }
  feature {
    key: "capital_loss"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "education"
    value {
      bytes_list {
        value: "Bachelors"
      }
    }
  }
  feature {
    key: "education_num"
    value {
      int64_list {
        value: 13
      }
    }
  }
  feature {
    key: "fnlwgt"
    value {
      int64_list {
        value: 77516
      }
    }
  }
  feature {
    key: "gender"
    value {
      bytes_list {
        value: "Male"
      }
    }
  }
  feature {
    key: "hours_per_week"
    value {
      int64_list {
        value: 40
      }
    }
  }
  feature {
    key: "income_bracket"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "marital_status"
    value {
      bytes_lis

## 2. StatisticsGen

In [11]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Generating statistics for split train
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/StatisticsGen/statistics/2/eval.
/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/StatisticsGen/statistics/2, id: 3)]
        ))

In [12]:
%%skip_for_export

context.show(statistics_gen.outputs['statistics'])


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


This cell will be skipped during export to pipeline.


### SchemaGen

In [13]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False
)
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Infering schema from statistics.
INFO:absl:Schema written to /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/SchemaGen/schema/3, id: 4)]
        ))

In [14]:
schema_gen.outputs['schema']

Channel(
    type_name: Schema
    artifacts: [Artifact(type_name: Schema, uri: /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/SchemaGen/schema/3, id: 4)]
)

In [15]:
%%skip_for_export

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,single,-
'capital_gain',INT,required,single,-
'capital_loss',INT,required,single,-
'education',STRING,required,single,'education'
'education_num',INT,required,single,-
'fnlwgt',INT,required,single,-
'gender',STRING,required,single,'gender'
'hours_per_week',INT,required,single,-
'income_bracket',INT,required,single,-


,Values
Domain,
'education',"'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
'gender',"'Female', 'Male'"
'marital_status',"'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
'native_country',"'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'"
'occupation',"'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
'race',"'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'"
'relationship',"'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
'workclass',"'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"


This cell will be skipped during export to pipeline.


### Validator

In [16]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics.
INFO:absl:Validation complete. Anomalies written to /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/ExampleValidator/anomalies/4.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/ExampleValidator/anomalies/4, id: 5)]
        ))

In [17]:
%%skip_for_export

context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'native_country',Unexpected string values,Examples contain values missing from the schema: Holand-Netherlands (<1%).


This cell will be skipped during export to pipeline.


## 3. Transform 

### Census Data

In [18]:
_binary_constants_module_file = 'binary_constants.py'

In [19]:
%%skip_for_export
%%writefile {_binary_constants_module_file}
CATEGORICAL_FEATURES = ['workclass', 'education', 'marital_status', 'occupation', 'relationship',
    'race', 'gender', 'native_country']

NUMERIC_FEATURES = [ 'age', 'education_num', 'capital_gain', 'capital_loss',
    'hours_per_week', 'fnlwgt']

# Number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
OOV_SIZE = 10

LABEL_KEY = 'income_bracket'

def transformed_name(key):
    return key + '_xf'



Overwriting binary_constants.py
This cell will be skipped during export to pipeline.


In [20]:
_binary_transform_module_file = 'binary_transform.py'

In [21]:
%%skip_for_export
%%writefile {_binary_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft
import binary_constants
import importlib

importlib.reload(binary_constants)

NUMERIC_FEATURES = binary_constants.NUMERIC_FEATURES
CATEGORICAL_FEATURES = binary_constants.CATEGORICAL_FEATURES
LABEL_KEY = binary_constants.LABEL_KEY
#VOCAB_SIZE = binary_constants.VOCAB_SIZE
#OOV_SIZE = binary_constants.OOV_SIZE
#TRANSFORMED_NAME = binary_constants.transformed_name

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
      Args:
        inputs: map from feature keys to raw not-yet-transformed features.
      Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = inputs.copy()
    
    #Scale numeric columns to have range [0,1] - if Tensors.
    for key in NUMERIC_FEATURES:
        #outputs[key] = tft.scale_to_0_1(inputs[key])
        #outputs[transformed_name(key)] = tft.scale_to_z_score(_fill_in_missing(inputs[key]))
        outputs[key] = tft.scale_to_0_1(_fill_in_missing(outputs[key]))
    
    # For all categorical columns except the label column, we generate a
    # vocabulary but do not modify the feature.  This vocabulary is instead
    # used in the trainer, by means of a feature column, to convert the feature
    # from a string to an integer id.
    
    for key in CATEGORICAL_FEATURES:
        outputs[key] = tft.compute_and_apply_vocabulary(
                                            _fill_in_missing(outputs[key]),
                                            #top_k=VOCAB_SIZE,
                                            #num_oov_buckets=OOV_SIZE
                                            )
    outputs[LABEL_KEY] = _fill_in_missing(outputs[LABEL_KEY])
    
    return outputs

def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.
    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
    Args:
        x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
          in the second dimension.
        Returns:
        A rank 1 tensor where missing values of `x` have been filled in.
    """     
    def_value = '' if x.dtype == tf.string else 0
    return tf.squeeze(
          tf.sparse.to_dense(
              tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]), def_value), axis=1)

Overwriting binary_transform.py
This cell will be skipped during export to pipeline.


In [ ]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=_binary_transform_module_file
)
context.run(transform)

In [23]:
transform.outputs

{'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/Transform/transform_graph/5, id: 6)]
), 'transformed_examples': Channel(
    type_name: Examples
    artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/Transform/transformed_examples/5, id: 7)]
)}

In [24]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transform_fn', 'metadata', 'transformed_metadata']

In [25]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "age"
    value {
      float_list {
        value: 0.301369845867157
      }
    }
  }
  feature {
    key: "capital_gain"
    value {
      float_list {
        value: 0.02174021676182747
      }
    }
  }
  feature {
    key: "capital_loss"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "education"
    value {
      int64_list {
        value: 2
      }
    }
  }
  feature {
    key: "education_num"
    value {
      float_list {
        value: 0.8000000715255737
      }
    }
  }
  feature {
    key: "fnlwgt"
    value {
      float_list {
        value: 0.044301897287368774
      }
    }
  }
  feature {
    key: "gender"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "hours_per_week"
    value {
      float_list {
        value: 0.3979591727256775
      }
    }
  }
  feature {
    key: "income_bracket"
    value {
      int64_list {
        value: 0
      }
    }
  }
 

# 4. Training

## Estimator Training

In [ ]:
trainer = Trainer(
      module_file=_binary_trainer_module_file,
      examples=transform.outputs['transformed_examples'],
      schema=schema_gen.outputs['schema'],
      #schema=infer_schema.outputs['schema'],
      #base_models=latest_model_resolver.outputs['latest_model'],
      transform_graph=transform.outputs['transform_graph'],
      train_args=trainer_pb2.TrainArgs(num_steps=1000),
      eval_args=trainer_pb2.EvalArgs(num_steps=500) 
    )

context.run(trainer)

## Functional Training

In [26]:
functional_binary_trainer = 'functional_binary_trainer.py'

In [27]:
trainer = Trainer(
    module_file=os.path.abspath(functional_binary_trainer),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=500) 
)

context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Training model.


Your model has been built....
Train for 1000 steps, validate for 500 steps
Epoch 1/5
1000/1000 [==============================] - 10s 10ms/step - loss: 0.4403 - accuracy: 0.7920 - val_loss: 0.3956 - val_accuracy: 0.8134
Epoch 2/5
1000/1000 [==============================] - 12s 12ms/step - loss: 0.3665 - accuracy: 0.8315 - val_loss: 0.3611 - val_accuracy: 0.8309
Epoch 3/5
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3452 - accuracy: 0.8400 - val_loss: 0.3507 - val_accuracy: 0.8404
Epoch 4/5
1000/1000 [==============================] - 10s 10ms/step - loss: 0.3361 - accuracy: 0.8437 - val_loss: 0.3497 - val_accuracy: 0.8418
Epoch 5/5
1000/1000 [==============================] - 8s 8ms/step - loss: 0.3303 - accuracy: 0.8478 - val_loss: 0.3426 - val_accuracy: 0.8414
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/Trainer/model/6/serving_model_d

INFO:absl:Training complete. Model written to /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/Trainer/model/6/serving_model_dir
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/Trainer/model/6, id: 8)]
        ))

In [28]:
%%skip_for_export

model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'serving_model_dir')
pp.pprint(os.listdir(model_dir))

['serving_model_dir']
['assets', 'variables', 'saved_model.pb']
This cell will be skipped during export to pipeline.


In [89]:
#%%skip_for_export

#log_dir = os.path.join(model_artifact_dir, 'logs')

#%load_ext tensorboard
#%tensorboard --logdir {log_dir}

This cell will be skipped during export to pipeline.


## EvalConfig

In [30]:
eval_config = tfma.EvalConfig(
    model_specs=[
        # Using signature 'eval' implies the use of an EvalSavedModel. To use
        # a serving model remove the signature to defaults to 'serving_default'
        # and add a label_key.
        #tfma.ModelSpec(signature_name='eval')
        tfma.ModelSpec(label_key='income_bracket')

    ],
    metrics_specs=[
        tfma.MetricsSpec(
            
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount')
            ],
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            thresholds = {
                'accuracy': tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.5})
                    ,
                    change_threshold=tfma.GenericChangeThreshold(
                       direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                       absolute={'value': -1e-10})
                )
            }
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        #tfma.SlicingSpec(feature_keys=['age'])
    ]
)

In [32]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))
context.run(model_resolver)

INFO:absl:Running driver for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ResolverNode.latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
        ))

In [ ]:

# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    # Change threshold will be ignored if there is no baseline (first run).
    eval_config=eval_config)
context.run(evaluator)

In [34]:
%%skip_for_export

evaluator.outputs

{'evaluation': Channel(
    type_name: ModelEvaluation
    artifacts: [Artifact(type_name: ModelEvaluation, uri: /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/Evaluator/evaluation/9, id: 9)]
), 'blessing': Channel(
    type_name: ModelBlessing
    artifacts: [Artifact(type_name: ModelBlessing, uri: /tmp/tfx-interactive-2020-07-16T10_55_53.876451-45zrh4eh/Evaluator/blessing/9, id: 10)]
)}

This cell will be skipped during export to pipeline.


In [ ]:
%%skip_for_export

context.show(evaluator.outputs['evaluation'])

In [ ]:
%%skip_for_export

import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

#tfma.view.render_plot(
#    tfma_result,
#    slicing_spec=None,
    #label="V1"
#)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, 
    slicing_column='age'
)

In [ ]:
print('TFMA version: {}'.format(tfma.version.VERSION_STRING))


In [ ]:
%%skip_for_export

blessing_uri = evaluator.outputs.blessing.get()[0].uri
!ls -l {blessing_uri}

In [ ]:
%%skip_for_export

PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

## Pusher

In [ ]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized


In [126]:
%%skip_for_export

pusher.outputs

{'pushed_model': Channel(
    type_name: PushedModel
    artifacts: [Artifact(type_name: PushedModel, uri: /tmp/tfx-interactive-2020-07-15T12_08_38.704975-m3a52mnx/Pusher/pushed_model/61, id: 68)]
)}

This cell will be skipped during export to pipeline.


In [ ]:
%%skip_for_export

push_uri = pusher.outputs.pushed_model.get()[0].uri
#push_uri = pusher.outputs.model_push.get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)